In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from statistics import mean

In [3]:
results_folder = os.getcwd()
csv_details = "EXPERIMENTS-LAB.csv"
csv_explicit = "explicit.csv"
csv_implicit = "implicit.csv"
csv_init = "init.csv"

In [4]:
exp_details = pd.read_csv(results_folder+"/"+csv_details)
survey_explicit = pd.read_csv(results_folder+"/"+csv_explicit)
survey_implicit = pd.read_csv(results_folder+"/"+csv_implicit)
survey_init = pd.read_csv(results_folder+"/"+csv_init)

Function successfully; Act consistently; Reliable; Predictable; Dependable; Follow directions; Meet the needs of the mission; Perform exactly as instructed; Have errors (Reverse Coded); Provide appropriate information; Unresponsive (Reverse Coded); Malfunction (Reverse Coded); Communicate with people; Provide feedback

In [4]:
def get_trust_scores(df_survey, from_col):
    trust_scores = []
    to_col = from_col+14
    
    for index in range(len(df_survey)):
        all_values = df_survey.iloc[index].tolist()

        trust_subscale = all_values[from_col:to_col] # 14 items
        # From percentage (xx%) to float (0.xx)
        for i in range(len(trust_subscale)):
            trust_subscale[i] = float(trust_subscale[i].replace('%', 'e-2'))

        # REVERSE: Have errors (Reverse Coded); Unresponsive (Reverse Coded); Malfunction (Reverse Coded)
        for i in range(1, 4):
            trust_subscale[-i] = 1-trust_subscale[-i]

        trust_scores.append(np.round(mean(trust_subscale), 2))
        
    return trust_scores

In [5]:
trust_init_scores = get_trust_scores(survey_init, 3)
trust_implicit_scores = get_trust_scores(survey_implicit, 1)
trust_explicit_scores = get_trust_scores(survey_explicit, 1)

# Create list of first and second surveys (explicit, implicit)
first_surveys = []
second_surveys = []
for index in np.unique(exp_details['ID']):
    order_behaviors = exp_details[exp_details['ID'] == index]['BEHAVIOR'].tolist()
    first_surveys.append(order_behaviors[0])
    second_surveys.append(order_behaviors[1])

# Create list with trust scores based on survey done
first_trust = []
second_trust = []
for i in range(len(first_surveys)):
    if first_surveys[i] == 'explicit' and second_surveys[i] == 'implicit':
        first_trust.append(trust_explicit_scores[i])
        second_trust.append(trust_implicit_scores[i])
    elif first_surveys[i] == 'implicit' and second_surveys[i] == 'explicit':
        first_trust.append(trust_implicit_scores[i])
        second_trust.append(trust_explicit_scores[i])
    else:
        print("ERROR: in explicit or implicit trust sequence.")

In [6]:
df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )
df['INIT_TRUST'] = trust_init_scores
df['FIRST_SURVEY'] = first_surveys
df['FIRST_TRUST'] = first_trust
df['SECOND_SURVEY'] = second_surveys
df['SECOND_TRUST'] = second_trust
df

,ID,INIT_TRUST,FIRST_SURVEY,FIRST_TRUST,SECOND_SURVEY,SECOND_TRUST
0,0,0.76,explicit,0.89,implicit,0.95
1,1,0.86,implicit,0.81,explicit,0.89
2,2,0.84,explicit,0.65,implicit,0.85
3,3,0.81,implicit,0.89,explicit,0.79
4,4,0.64,explicit,0.97,implicit,0.89
...,...,...,...,...,...,...
57,57,0.80,implicit,0.94,explicit,0.96
58,58,0.75,explicit,0.75,implicit,0.83
59,59,0.59,implicit,0.14,explicit,0.12
60,60,0.81,explicit,0.80,implicit,0.78


In [10]:
df.to_csv("results_trust.csv")

## ANALYSIS

In [5]:
df = pd.read_csv("results_trust.csv")

In [6]:
from scipy import stats

stats.kstest(df["INIT_TRUST"].tolist(), df["FIRST_TRUST"].tolist())

KstestResult(statistic=0.3064516129032258, pvalue=0.005655400014355446)

In [12]:
print("Init trust")
print("Mean: ", np.round(mean(trust_init_scores), 2))
print("STD: ", np.round(np.std(trust_init_scores), 2))

Init trust
Mean:  0.77
STD:  0.11


In [13]:
print("First trust")
print("Mean: ", np.round(mean(first_trust), 2))
print("STD: ", np.round(np.std(first_trust), 2))

First trust
Mean:  0.83
STD:  0.15


In [9]:
print("Second trust")
print("Mean: ", np.round(mean(second_trust), 2))
print("STD: ", np.round(np.std(second_trust), 2))

Second trust
Mean:  0.84
STD:  0.16
